In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
iris=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

In [3]:
iris.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
mnist=pd.read_csv('https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/mnist_test.csv',header=None)

In [5]:
mnist.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**IRIS**

Let’s identify a portion of our data for which we will try to guess the species. Randomly select 20%
of the records and store the first four columns (i.e. the features representing each flower) into a
two-dimensional numpy array of shape N × C, you can call it X_test.
For the same records, store the last column (i.e. the one with the species values) into another array,
namely y_test. This is the data that will be used to test the accuracy of your KNN implementation
and its correct functioning (i.e. the testing data).

In [6]:
iris.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
values=iris.values
X=iris.iloc[:,:4]
y=iris[4]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) #split the data

In [9]:
X_train.shape

(120, 4)

In [242]:
class KNearestNeighbors:
    

    def __init__(self, k, distance_metric="euclidean",weights="unifrom"):
        self.k = k
        self.distance_metric = distance_metric
        self.X=None
        self.y=None
        self.y_pred=[]
        self.weights=weights

    def sort_vector(a,order):
        temp=[]
        i=0
        while i<=3:
            temp.append(a[order[i]-1])
            i+=1

        return temp

    def fit(self, X, y):
        from sklearn.preprocessing import MinMaxScaler #We need this to normalize the data
        X_size = X.shape
        #mm=MinMaxScaler()
        #self.X,self.y=mm.fit_transform(X,y)
        self.X=X.values
        self.y=y.values
        return self

        
    def predict(self, X):


        from scipy.spatial.distance import cdist as d

        if self.distance_metric=="euclidean":
            distance=d(X,self.X, metric='euclidean')
        elif self.distance_metric== "cosine":
            distance=d(X,self.X, metric='cosine')
        else:
            distance=d(X,self.X, metric='cityblock')
    
        d_m=np.argsort(distance,axis=1) #These are the indeces of the distances ordered 
        d_m=d_m[:,:self.k]#Now I have to select th k similar indeces

        #Select the position in the y_train (the flower)
        flowers=[]

        #self.y=self.y.values

        for i in range(len(d_m)):
            a=list(map(lambda x: self.y[x],d_m[i]))
            flowers.append(a) #Here I have the k similar flowers

        #The most common flower
        #When the distance is the same we will put some weights to the neighbors 
        if self.weights==self.distance_metric:
            #The weights must be a matrix sucha as the columns are the weights of the neigbords and the rows the X_test
            self.weights=1/distance[:,:self.k]
            #self.weights= self.sort_vector(self.weights,d_m) #The weights are ordered by the k index distacnces
            #Now I select the flower with the greatest weight
            index=np.argsort(self.weights,axis=1)[:,::-1][:,:1] #The indeces of the biggest weight
            #Now we will create the prediction
            for i in range(len(flowers)):
                self.y_pred.append(flowers[i][index[i][0]])


            
        # Just if weights="uniform"
        else:

            for i in flowers:

                count0=0
                count1=1
                count2=2

                for j in range(3):

                    if i[j]=='Iris-setosa':
                        count0+=1
                    elif i[j]=='Iris-versicolor':
                        count1+=1
                    elif i[j]=='Iris-virginica':
                        count2+=1

                if count0>count1:

                    if count0>count2:
                        self.y_pred.append('Iris-setosa')
                    else:
                        self.y_pred.append('Iris-virginica')

                elif count1>count2:

                    self.y_pred.append('Iris-versicolor')
                
                else:

                    self.y_pred.append('Iris-virginica')

        return self.y_pred#flowers,index, self.weights, distance
      



In [11]:
def ed(p,q,n):
    sum=0
    for i in range(n):
        sum+=(p[i]-q[i])**2
    distance=np.sqrt(sum)
    return distance

In [12]:
def cd(p,q,n):
    sumq=0
    sump=0
    sumpq=0
    for i in range(n):
        sumq+=q[i]**2
        sump+=p[i]**2
        sumpq+=p[i]*q[i]
    cs=sumpq/(sump*sumq)
    return cs

In [13]:
def md(p,q,n):
    md=0
    for i in range(n):
        md+=np.abs(p[i]-q[i])
    return md

In [243]:
knn=KNearestNeighbors(k=3,weights="euclidean")

In [244]:
knn.fit(X_train,y_train)

In [245]:
knn.predict(X_test)

['Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-virginica']

In [250]:
mnist.shape

(10000, 785)